Notebook for running the project pipeline and experiments separately

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys, os

# Path to the folder where utils.py is located
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/CS7643 Project/OMSCS_7643_F2025_Project/'

# Add to Python's import search path
sys.path.append(GOOGLE_DRIVE_PATH)

# Check that utils.py is visible
print("Files in path:", os.listdir(GOOGLE_DRIVE_PATH))

# Now import the helper functions


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
%cd $GOOGLE_DRIVE_PATH


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
# from scripts.Trans import train_transformer, predict, train_classifier
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
# os.chdir(Path().resolve().parent)
print("Current working directory:", os.getcwd())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python scripts/preprocess.py

In [ ]:
!python scripts/embedding.py

In [ ]:
!python scripts/build_features.py --lookback 24

In [ ]:
# ## Test Build features
# import os
# import numpy as np
# import pandas as pd
# from pathlib import Path

# import importlib.util

# spec = importlib.util.spec_from_file_location(
#     "build_features", "scripts/build_features.py"
# )
# build_features = importlib.util.module_from_spec(spec)
# spec.loader.exec_module(build_features)

# # ===== Test Settings =====
# FEATURE_DIR = Path("output/features")
# LOOKBACK = 24
# MODES = ["mean", "volume", "exp_decay", "max", "attn"]


# def run_one_test(mode):
#     print(f"\n\n==============================")
#     print(f"TEST: lookback_mode = {mode}")
#     print(f"==============================")

#     # delete old files
#     for f in ["X.npy", "y.npy", "dataset.parquet"]:
#         fp = FEATURE_DIR / f
#         if fp.exists():
#             fp.unlink()

#     # set args
#     import sys
#     sys.argv = [
#         "build_features.py",
#         "--lookback", str(LOOKBACK),
#         "--lookback-mode", mode,
#     ]

#     # 运行 main()
#     try:
#         build_features.main()
#     except Exception as e:
#         print("Script crashed:", e)
#         return False

#     # check files
#     x_file = FEATURE_DIR / "X.npy"
#     y_file = FEATURE_DIR / "y.npy"
#     df_file = FEATURE_DIR / "dataset.parquet"

#     if not x_file.exists():
#         print("Missing X.npy")
#         return False
#     if not y_file.exists():
#         print("Missing y.npy")
#         return False
#     if not df_file.exists():
#         print("Missing dataset.parquet")
#         return False

#     # load data
#     X = np.load(x_file)
#     Y = np.load(y_file)
#     df = pd.read_parquet(df_file)

#     print("Shapes:", X.shape, Y.shape)

#     if len(X) == 0 or len(Y) == 0:
#         print("Empty X or y!")
#         return False

#     if X.shape[0] != Y.shape[0]:
#         print("X and y length mismatch!")
#         return False

#     if X.ndim != 2:
#         print("X must be 2D")
#         return False

#     if Y.ndim != 1:
#         print("y must be 1D")
#         return False

#     if df.shape[0] != X.shape[0]:
#         print("dataset.parquet row mismatch!")
#         return False

#     print("Files validated OK")
#     return True



# # ===== Run all tests =====
# print("\n=== Running Jupyter Test Suite for build_features.py ===")

# passed = 0

# for m in MODES:
#     ok = run_one_test(m)
#     if ok:
#         print(f"PASS: {m}")
#         passed += 1
#     else:
#         print(f"FAIL: {m}")

# print("\n==============================")
# print(f"Final Summary: {passed}/{len(MODES)} tests passed")
# print("==============================")


In [ ]:
!python scripts/train.py --model lr --seq_len 1 --epochs 30

In [ ]:
!python scripts/predict.py --model lr --seq_len 1

In [ ]:
!python scripts//train.py --model transformer --seq_len 12 \
    --model transformer \
    --batch_size 64 \
    --lr 5e-5 \
    --epochs 50 \
    --weight_decay 1e-4 \
    --grad_clip 1.0 \
    --scheduler cosine_warmup \
    --warmup_pct 0.1 \
    --ema_decay 0.995 \
    --seed 42

In [ ]:
!python scripts/predict.py --model transformer --seq_len 12

In [ ]:
!python scripts/train.py --model gru --num_layers 3



In [ ]:
!python scripts/evaluate.py --predictions output/predictions/predictions_transformer.csv

In [ ]:
##Check Data used for trainning
df = pd.read_parquet("output/features/dataset.parquet")
df.tail()

Board Based First Round Tuning (Sub rounds 1-2 are experimental trails, and 3-5 are used as base for the second round)

(Sub)First Round Tuning - 24 hours lookback without changing feature aggregation

In [ ]:
from scripts.hypersearch import HyperSearch

hs = HyperSearch(
    max_runs=2000,
    search_mode="full"   # or "quick"
)
hs.run()

First Round Tuning Best Model

In [ ]:
import json
## The original file location has been moved to output1128 folder
best_dir = "/content/drive/MyDrive/CS7643 Project/OMSCS_7643_F2025_Project/output/hyper_runs/run_0103"

with open(best_dir + "/config.json") as f:
    config = json.load(f)
config


In [ ]:
!python scripts/train.py \
 --model gru \
 --seq_len 8 \
 --batch_size 64 \
 --lr 0.0003 \
 --epochs 30 \
 --weight_decay 0.0 \
 --warmup_pct 0.1 \
 --ema_decay 0.995 \
 --hidden_dim 512


In [ ]:
!python scripts/predict.py \
 --model gru \
 --seq_len 8 \
 --batch_size 64 \
 --cutoff_date 2024-10-01

In [ ]:
!python scripts/evaluate.py \
 --predictions output/predictions/predictions_gru.csv

(Sub)Second round tuning ++ Feature Aggregation Parameters

In [ ]:
from scripts.hypersearch import HyperSearch

hs = HyperSearch(
    max_runs=30000,
    search_mode="full"   # or "quick"
)
hs.run()

In [ ]:
import json
from pathlib import Path
import pandas as pd

RUN_DIR = Path("/content/drive/MyDrive/CS7643 Project/OMSCS_7643_F2025_Project/output/hyper_runs")

rows = []

for run_dir in sorted(RUN_DIR.iterdir()):
    config_path = run_dir / "config.json"
    result_path = run_dir / "metrics.json"

    if not config_path.exists() or not result_path.exists():
        continue

    try:
        config = json.load(open(config_path))
        metrics = json.load(open(result_path))
    except:
        continue

    pred = metrics.get("model_predictions", {})
    subsets = pred.get("subset_metrics", {})

    # ============ TEST METRICS  ============
    test = subsets.get("test", {})
    test_reg = test.get("regression_metrics", {})
    test_strat = test.get("strategy_metrics", {})

    test_ic = test_reg.get("pearson_ic", None)
    test_sharpe = test_strat.get("sharpe", None)

    # ============ HOLDOUT METRICS  ============
    hold = subsets.get("holdout", {})
    hold_reg = hold.get("regression_metrics", {})
    hold_strat = hold.get("strategy_metrics", {})

    hold_ic = hold_reg.get("pearson_ic", None)
    hold_sharpe = hold_strat.get("sharpe", None)

    rows.append({
        "run": run_dir.name,
        "model": config.get("model"),
        "test_IC": test_ic,
        "test_Sharpe": test_sharpe,
        "holdout_IC": hold_ic,
        "holdout_Sharpe": hold_sharpe,
        "config": config
    })

df = pd.DataFrame(rows)

print("\n=== Run Count Per Model ===")
print(df["model"].value_counts())

# ===============================================================
# === APPLY HYPERSEARCH LOGIC: test_Sharpe > 0 & maximize test_IC
# ===============================================================
df_valid = df[df["test_Sharpe"] > 0]   # HyperSearch willfilter Sharpe≤0 runs

print("\n=== Best Run Per Model (HyperSearch Logic: test_sharpe>0 & max test_IC) ===")
best_runs = df_valid.sort_values("test_IC", ascending=False).groupby("model").head(1)

print(best_runs[[
    "model",
    "run",
    "test_IC",
    "test_Sharpe",
    "holdout_IC",
    "holdout_Sharpe"
]])



In [ ]:
## Check round 2 logs
# import json
# ## The original file location has been moved to output1128 folder
# best_dir = "/content/drive/MyDrive/CS7643 Project/OMSCS_7643_F2025_Project/output/hyper_runs_2nd_round/run_0332"

# with open(best_dir + "/config.json") as f:
#     config = json.load(f)
# config

Reproducing the best IC models

In [ ]:
##GRU
!python scripts/build_features.py --horizon 3 --lookback 48 --lookback-mode exp_decay
!python scripts/train.py --model gru --seq_len 12 --batch_size 128 --lr 3e-05 --epochs 15 --weight_decay 0.0 --warmup_pct 0.05 --hidden_dim 256 --lstm_proj_dim 32 --num_layers 2 --tf_d_model 512 --tf_heads 4 --tf_layers 4 --tf_ff_dim 256 --tf_pool attention --tf_dropout 0.1
!python scripts/predict.py --model gru --seq_len 12 --batch_size 128 --cutoff_date 2024-10-01
!python scripts/evaluate.py --include-holdout --predictions output/predictions/predictions_gru.csv

In [ ]:
## Transformer
!python scripts/build_features.py --horizon 3 --lookback 24 --lookback-mode attn
!python scripts/train.py --model transformer --seq_len 8 --batch_size 128 --lr 0.0001 --epochs 30 --weight_decay 0.0 --scheduler cosine_warmup --warmup_pct 0.1 --hidden_dim 768 --lstm_proj_dim 64 --num_layers 2 --tf_d_model 384 --tf_heads 2 --tf_layers 2 --tf_ff_dim 256 --tf_pool attention --tf_dropout 0.1
!python scripts/predict.py --model transformer --seq_len 8 --batch_size 128 --cutoff_date 2024-10-01
!python scripts/evaluate.py --include-holdout --predictions output/predictions/predictions_transformer.csv

In [ ]:
## MLP
!python scripts/build_features.py --horizon 3 --lookback 6 --lookback-mode volume
!python scripts/train.py --model mlp --seq_len 1 --batch_size 64 --lr 3e-05 --epochs 15 --weight_decay 0.0001 --scheduler cosine_warmup --warmup_pct 0.05 --hidden_dim 256 --lstm_proj_dim 32 --num_layers 2 --tf_d_model 512 --tf_heads 4 --tf_layers 4 --tf_ff_dim 1024 --tf_pool cls --tf_dropout 0.1
!python scripts/predict.py --model mlp --seq_len 1 --batch_size 64 --cutoff_date 2024-10-01
!python scripts/evaluate.py --include-holdout --predictions output/predictions/predictions_mlp.csv

In [ ]:
## LSTM
!python scripts/build_features.py --horizon 1 --lookback 12 --lookback-mode max
!python scripts/train.py --model lstm --seq_len 12 --batch_size 128 --lr 0.0001 --epochs 30 --weight_decay 1e-05 --warmup_pct 0.1 --hidden_dim 256 --lstm_proj_dim 256 --num_layers 2 --tf_d_model 512 --tf_heads 8 --tf_layers 2 --tf_ff_dim 512 --tf_pool cls --tf_dropout 0.1
!python scripts/predict.py --model lstm --seq_len 12 --batch_size 128 --cutoff_date 2024-10-01
!python scripts/evaluate.py --include-holdout --predictions output/predictions/predictions_lstm.csv

(Sub)3-5 Rounds Tuning ++ More Horizon and Lookback

In [ ]:
from scripts.hypersearch import HyperSearch

hs = HyperSearch(
    max_runs=30000,
    search_mode="full"   # or "quick"
)
hs.run()

In [ ]:
import pandas as pd
from pathlib import Path
import json

# Paths
BASE = Path("/content/drive/MyDrive/CS7643 Project/OMSCS_7643_F2025_Project/output/Tuning")
SUMMARY_PATH = BASE / "Tuning_Summary.xlsx"

# Folder mapping based on round number
ROUND_TO_FOLDER = {
    3: BASE / "hyper_runs_3rd_round",
    4: BASE / "hyper_runs_4th_round",
    5: BASE / "hyper_runs_5th_round"
}

# Load summary file
df = pd.read_excel(SUMMARY_PATH)

# Prepare new columns
df["IC_holdout"] = None
df["Sharpe_holdout"] = None

def load_metrics(metrics_path):
    """Load metrics.json safely."""
    try:
        with open(metrics_path, "r") as f:
            return json.load(f)
    except:
        return None

# Process each row
for idx, row in df.iterrows():
    round_num = int(row["Round"])
    run_id = int(row["run_id"])

    # Determine run folder
    run_folder = ROUND_TO_FOLDER[round_num] / f"run_{run_id:04d}"
    metrics_path = run_folder / "metrics.json"

    if not metrics_path.exists():
        print(f"[Warning] Missing metrics.json: {metrics_path}")
        continue

    metrics = load_metrics(metrics_path)
    if metrics is None:
        print(f"[Warning] Could not load metrics: {metrics_path}")
        continue

    # Extract holdout metrics
    subset = metrics.get("model_predictions", {}).get("subset_metrics", {})
    hold = subset.get("holdout", {})

    hold_reg = hold.get("regression_metrics", {})
    hold_strat = hold.get("strategy_metrics", {})

    ic = hold_reg.get("pearson_ic", None)
    sharpe = hold_strat.get("sharpe", None)

    df.at[idx, "IC_holdout"] = ic
    df.at[idx, "Sharpe_holdout"] = sharpe

    print(f"Round {round_num} run {run_id} → Holdout IC={ic}, Sharpe={sharpe}")

# Save updated file
output_path = BASE / "Tuning_Summary_with_holdout.xlsx"
df.to_excel(output_path, index=False)

print(f"\nSaved: {output_path}")


Reproduce best models round 3,4,5

In [ ]:
## Best Avg SR across Test and Holdout
# !python scripts/build_features.py --horizon 1 --lookback 12 --lookback-mode attn
# !python scripts/train.py --model lstm --seq_len 720 --batch_size 128 --lr 0.0001 --epochs 15 --weight_decay 0.0001 --warmup_pct 0.05 --hidden_dim 768 --lstm_proj_dim 128 --num_layers 3 --tf_d_model 384 --tf_heads 2 --tf_layers 4 --tf_ff_dim 512 --tf_pool cls --tf_dropout 0.2
# !python scripts/predict.py --model lstm --seq_len 720 --batch_size 128 --cutoff_date 2024-10-01
# !python scripts/evaluate.py --include-holdout --predictions output/predictions/predictions_lstm.csv

In [ ]:
## Best Avg SR across Test and Holdout
# !python scripts/build_features.py --horizon 1 --lookback 6 --lookback-mode mean
# !python scripts/train.py --model gru --seq_len 12 --batch_size 128 --lr 0.001 --epochs 15 --weight_decay 1e-05 --scheduler cosine_warmup --warmup_pct 0.05 --hidden_dim 768 --lstm_proj_dim 32 --num_layers 3 --tf_d_model 384 --tf_heads 8 --tf_layers 4 --tf_ff_dim 256 --tf_pool cls --tf_dropout 0.1
# !python scripts/predict.py --model gru --seq_len 12 --batch_size 128 --cutoff_date 2024-10-01
# !python scripts/evaluate.py --include-holdout --predictions output/predictions/predictions_gru.csv

Second round model wise tuning - for each key paramter for the model one by one

In [ ]:
!python scripts/hypersearch2.py --mode modelwise --max-runs 2000

In [ ]:
!python scripts/model_select.py

Back up for commit

In [ ]:
!zip -r project_backup.zip "/content/drive/MyDrive/CS7643 Project/OMSCS_7643_F2025_Project" -x "*/__pycache__/*"

In [ ]:
from google.colab import files
files.download("project_backup.zip")

#### early poc of the ppl in ipynb below

In [ ]:
news_path = "/content/drive/MyDrive/CS7643 Project/OMSCS_7643_F2025_Project/data/cryptonews.csv"
df = pd.read_csv(news_path)
## Test with sample data
# df = df[:1000]
df.head()

In [ ]:
## Text Data Preprocessing
if 'date' not in df.columns:
    raise ValueError("no 'date' column")

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])
df['content'] = (
    "Subject: " +
    df['subject'] + ". " +
    df['title'].fillna('') + ". " +
    df['text'].fillna('')
)
df = df[df['content'].str.strip() != ""]
df = df.reset_index(drop=True)
print("Example content:\n", df['content'].iloc[0])

In [ ]:
## Finbert embedding
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert").to(device)
model.eval()

def get_cls_embedding(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    cls_emb = outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()
    return cls_emb

emb_list = []

print("Generating CLS embeddings with subject-aware input…")
for text in tqdm(df['content'], total=len(df)):
    emb = get_cls_embedding(text)
    emb_list.append(emb)

emb_matrix = np.vstack(emb_list)   # (N, 768)


In [ ]:
## Hourly Text Embeddings
emb_cols = [f"emb_{i}" for i in range(768)]
emb_df = pd.DataFrame(emb_matrix, columns=emb_cols)
df_emb = pd.concat([df[['date']].reset_index(drop=True), emb_df], axis=1)
df_emb['date_hour'] = df_emb['date'].dt.floor('H') - pd.Timedelta(hours=1)
hourly_emb = df_emb.groupby('date_hour')[emb_cols].mean().reset_index()
hourly_emb.head()

In [ ]:
## Price Data
price_path = r"/content/drive/MyDrive/CS7643 Project/OMSCS_7643_F2025_Project/data/BTC_USD_hourly.parquet"
price_df = pd.read_parquet(price_path)
price_df['date_hour'] = pd.to_datetime(price_df['datetime'], errors='coerce').dt.tz_localize(None).dt.floor('H')
price_df = price_df.drop(columns=['datetime'])
price_df.head()

In [ ]:
df = price_df.merge(hourly_emb, on='date_hour', how='inner')
df = df.sort_values('date_hour').reset_index(drop=True)
print("Merged shape:", df.shape)

###**Predict Price**

In [ ]:
df['target_close'] = df['close'].shift(-1)
df = df.dropna().reset_index(drop=True)
df.head()

In [ ]:
price_cols = ['open', 'high', 'low', 'close']
emb_cols = [c for c in df.columns if c.startswith("emb_")]

feature_dim = len(price_cols) + len(emb_cols)
print("feature_dim =", feature_dim)

In [ ]:
total_len = len(df)
train_end = int(total_len * 0.7)

train_df = df.iloc[:train_end]
test_df  = df.iloc[train_end:]

sub_end = int(len(train_df) * 0.9)
subtrain_df = train_df.iloc[:sub_end]
val_df      = train_df.iloc[sub_end:]

In [ ]:
scaler_price = MinMaxScaler()
scaler_price.fit(subtrain_df[price_cols])

sub_price  = scaler_price.transform(subtrain_df[price_cols])
val_price  = scaler_price.transform(val_df[price_cols])
test_price = scaler_price.transform(test_df[price_cols])

# embedding
sub_emb  = subtrain_df[emb_cols].values * 0.01
val_emb  = val_df[emb_cols].values * 0.01
test_emb = test_df[emb_cols].values * 0.01

# combine
X_sub_df  = np.concatenate([sub_price,  sub_emb],  axis=1)
X_val_df  = np.concatenate([val_price,  val_emb],  axis=1)
X_test_df = np.concatenate([test_price, test_emb], axis=1)

# y scaler
scaler_y = MinMaxScaler()
scaler_y.fit(subtrain_df[['target_close']])

y_sub  = scaler_y.transform(subtrain_df[['target_close']]).reshape(-1)
y_val  = scaler_y.transform(val_df[['target_close']]).reshape(-1)
y_test = scaler_y.transform(test_df[['target_close']]).reshape(-1)


In [ ]:
def build_sequences(X_block, y_block, seq_len=12):
    X_list, y_list = [], []

    for i in range(seq_len, len(X_block)):
        X_list.append(X_block[i-seq_len:i])
        y_list.append(y_block[i])

    return np.array(X_list), np.array(y_list)


seq_len = 12

X_sub,  y_sub_seq  = build_sequences(X_sub_df,  y_sub,  seq_len)
X_val,  y_val_seq  = build_sequences(X_val_df,  y_val,  seq_len)
X_test, y_test_seq = build_sequences(X_test_df, y_test, seq_len)

print("Sub:", X_sub.shape, y_sub_seq.shape)
print("Val:", X_val.shape, y_val_seq.shape)
print("Test:", X_test.shape, y_test_seq.shape)


In [ ]:
X_train_all = np.concatenate([X_sub, X_val], axis=0)
y_train_all = np.concatenate([y_sub_seq, y_val_seq], axis=0)


model, train_losses, val_losses = train_transformer(
    X_train_all,
    y_train_all,
    batch_size=32,
    lr=1e-4,
    epochs=30
)

In [ ]:
# ---- test prediction ----
pred_norm = predict(model, X_test)           # (N_test,)
pred_price = scaler_y.inverse_transform(pred_norm.reshape(-1, 1)).reshape(-1)

# ---- ground truth ----
y_true_price = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).reshape(-1)


In [ ]:
plt.figure(figsize=(8,4))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Learning Curve")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
test_timestamps = test_df['date_hour'].values
test_seq_timestamps = test_timestamps[seq_len:]

plt.figure(figsize=(14,6))
plt.plot(test_seq_timestamps, y_true_price, label='True Price', linewidth=2)
plt.plot(test_seq_timestamps, pred_price, label='Predicted Price', linewidth=2)

plt.title("Predicted vs Actual Close Price (Real Timestamp)")
plt.xlabel("Time")
plt.ylabel("Price")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse_norm = mean_squared_error(y_test_seq, pred_norm)
mse_real = mean_squared_error(y_true_price, pred_price)

rmse = np.sqrt(mse_real)
mae  = np.mean(np.abs(y_true_price - pred_price))
mape = np.mean(np.abs((y_true_price - pred_price) / y_true_price)) * 100
acc  = 1 - mape/100   # vs paper

print("Normalized MSE:", mse_norm)
print("Real Price MSE:", mse_real)
print("RMSE:", rmse)
print("MAE:", mae)
print("MAPE(%):", mape)
print("Accuracy:", acc)


**Predict Retrurn**

In [ ]:
df['target_ret'] = df['close'].pct_change().shift(-1)
df = df.dropna().reset_index(drop=True)

In [ ]:
total_len = len(df)
train_end = int(total_len * 0.7)

train_df = df.iloc[:train_end]
test_df  = df.iloc[train_end:]

sub_end = int(len(train_df) * 0.9)
subtrain_df = train_df.iloc[:sub_end]
val_df      = train_df.iloc[sub_end:]
scaler_price = MinMaxScaler()
scaler_price.fit(subtrain_df[price_cols])

sub_price  = scaler_price.transform(subtrain_df[price_cols])
val_price  = scaler_price.transform(val_df[price_cols])
test_price = scaler_price.transform(test_df[price_cols])

sub_emb  = subtrain_df[emb_cols].values * 0.01
val_emb  = val_df[emb_cols].values * 0.01
test_emb = test_df[emb_cols].values * 0.01

X_sub_df  = np.concatenate([sub_price,  sub_emb],  axis=1)
X_val_df  = np.concatenate([val_price,  val_emb],  axis=1)
X_test_df = np.concatenate([test_price, test_emb], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_ret = StandardScaler()
scaler_ret.fit(subtrain_df[['target_ret']])

y_sub  = scaler_ret.transform(subtrain_df[['target_ret']]).reshape(-1)
y_val  = scaler_ret.transform(val_df[['target_ret']]).reshape(-1)
y_test = scaler_ret.transform(test_df[['target_ret']]).reshape(-1)

In [ ]:
X_sub,  y_sub_seq  = build_sequences(X_sub_df,  y_sub,  seq_len=12)
X_val,  y_val_seq  = build_sequences(X_val_df,  y_val,  seq_len=12)
X_test, y_test_seq = build_sequences(X_test_df, y_test, seq_len=12)

In [ ]:
X_train_all = np.concatenate([X_sub, X_val], axis=0)
y_train_all = np.concatenate([y_sub_seq, y_val_seq], axis=0)

model, train_losses, val_losses = train_transformer(
    X_train_all,
    y_train_all,
    lr=1e-4,
    batch_size=32,
    epochs=30
)

pred_norm = predict(model, X_test)
pred_ret = scaler_ret.inverse_transform(pred_norm.reshape(-1,1)).reshape(-1)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Learning Curve")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
residuals = y_true_price - pred_price

plt.figure(figsize=(10,4))
plt.plot(residuals)
plt.title("Residuals Over Time")
plt.ylabel("Residual (True - Pred)")
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

print("Return MSE:", mean_squared_error(y_test_seq, pred_norm))
print("Return R2:", r2_score(y_test_seq, pred_norm))

**Classify Up_Down**

In [ ]:
df['return'] = df['close'].pct_change()
df['label'] = (df['return'] > 0).astype(int)
df = df.dropna().reset_index(drop=True)

In [ ]:
total_len = len(df)
train_end = int(total_len * 0.7)

train_df = df.iloc[:train_end]
test_df  = df.iloc[train_end:]

sub_end = int(len(train_df) * 0.9)
subtrain_df = train_df.iloc[:sub_end]
val_df      = train_df.iloc[sub_end:]

price_cols = ['open', 'high', 'low', 'close']
emb_cols   = [c for c in df.columns if c.startswith("emb_")]

# --- price scaler ---
scaler_price = MinMaxScaler()
scaler_price.fit(subtrain_df[price_cols])

sub_price  = scaler_price.transform(subtrain_df[price_cols])
val_price  = scaler_price.transform(val_df[price_cols])
test_price = scaler_price.transform(test_df[price_cols])

# --- embedding (scale ×0.01) ---
sub_emb  = subtrain_df[emb_cols].values * 0.01
val_emb  = val_df[emb_cols].values * 0.01
test_emb = test_df[emb_cols].values * 0.01

# --- combine ---
X_sub_df  = np.concatenate([sub_price,  sub_emb],  axis=1)
X_val_df  = np.concatenate([val_price,  val_emb],  axis=1)
X_test_df = np.concatenate([test_price, test_emb], axis=1)

# --- labels (classification) ---
y_sub  = subtrain_df['label'].values
y_val  = val_df['label'].values
y_test = test_df['label'].values

In [ ]:
def build_sequences(X_block, y_block, seq_len=12):
    X_list, y_list = [], []

    for i in range(seq_len, len(X_block)):
        X_list.append(X_block[i-seq_len:i])
        y_list.append(y_block[i])

    return np.array(X_list), np.array(y_list)

seq_len = 12

X_sub,  y_sub_seq  = build_sequences(X_sub_df,  y_sub,  seq_len)
X_val,  y_val_seq  = build_sequences(X_val_df,  y_val,  seq_len)
X_test, y_test_seq = build_sequences(X_test_df, y_test, seq_len)

print("Shapes:")
print("Train:", X_sub.shape, y_sub_seq.shape)
print("Val:",   X_val.shape, y_val_seq.shape)
print("Test:",  X_test.shape, y_test_seq.shape)

In [ ]:
model = train_classifier(
    X_sub, y_sub_seq,
    X_val, y_val_seq,
    lr=1e-4,
    epochs=20,
    batch_size=32
)


In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

model.eval()
device = next(model.parameters()).device

with torch.no_grad():
    logits = model(torch.tensor(X_test, dtype=torch.float32).to(device))
    prob = torch.sigmoid(logits).cpu().numpy()
    pred_label = (prob > 0.5).astype(int)

acc = accuracy_score(y_test_seq, pred_label)
auc = roc_auc_score(y_test_seq, prob)

print("Test Accuracy:", acc)
print("Test AUC:", auc)
